In [ ]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Users/anura/anaconda3/Library/bin/graphviz'
import sys
import networkx as nx
import numpy as np
import random
import pickle
import sys
from numba import jit
import elfi
import sklearn as sk
import scipy
from scipy.spatial import distance

In [ ]:
seed = 20170530  # this will be separately given to ELFI
np.random.seed(seed)

In [ ]:
#parameters of the script

networktype = 'pref' #pref, smallworld, grid, ER, korea1, korea2, ckm
# nAgents = 4**2
nAgents = 10**2
sidelength = int(nAgents**.5) #10^2 = 100, 32^2 = 1024
haltMin = .6 # minimum % of nodes active
haltMax = .7 # maximum % of nodes active

pRewire = .1 # proportion of edges to rewire
numDatasets = 100 # number of data sets to generate
saveData = True #save the output to file?

In [ ]:
def testThresh(agents, mn, mx):
    if np.mean(agents) > mx:
        return False
    if np.mean(agents) < mn:
        return False
    if np.mean(agents) <= 0:
        return False
    if np.mean(agents) >= 1:
        return False

    return True


In [ ]:

def testThresh(agents, mn, mx):
    if np.mean(agents) > mx:
        return False
    if np.mean(agents) < mn:
        return False
    if np.mean(agents) <= 0:
        return False
    if np.mean(agents) >= 1:
        return False

    return True


def genNet(n, k=4, pRewire=.1, type='grid'):
  # create net
  if type == 'grid': #wrap the grid
    net = nx.grid_2d_graph(int(n**.5), int(n**.5), periodic=True)
    #net = nx.grid_2d_graph(3, 3, periodic=True)
    #net = nx.grid_2d_graph(2, 2, periodic=False)
    #plotNet(net)
    #print(len(net.edges()))
    #assert(0)
    # rewire
    numRewired = 0
    #while numRewired < (pRewire * nx.number_of_nodes(net)):
    while numRewired < 1:
      tries = 0
      while tries < 100:
        tries = tries + 1
        #print([numRewired, pRewire * nx.number_of_nodes(net), tries])
        v1 = random.choice(net.nodes())
        v2 = random.choice(net.nodes())
        if not( net.has_edge(v1,v2) or v1==v2 or len(net.neighbors(v1)) <= 1): #net.neighbors is sometimes (often?) a blank set, changed so v1 needs 2 nb
          #print net.neighbors(v1)
          break
      v1Neighbors = net.neighbors(v1)
      #print v1Neighbors
      #print v1
      #print v2
      #print(len(net.edges()))
      tobeDeleted = random.choice(v1Neighbors)
      net.remove_edge(v1, tobeDeleted)
      #print(len(net.edges()))
      #print([v1, tobeDeleted, v2])
      net.add_edge(v1, v2)
      numRewired = numRewired + 1
    #plotNet(net)
    #assert(0)
    return net, nx.to_numpy_matrix(net, dtype=np.float)

  elif type == 'smallworld':
    #net = nx.connected_watts_strogatz_graph(n, k, .15)
    net = nx.connected_watts_strogatz_graph(n, k, pRewire)
    return net, nx.to_numpy_matrix(net, dtype=np.float)  
  elif type == 'pref':
    net = nx.barabasi_albert_graph(n, 2)
    return net, nx.to_numpy_matrix(net, dtype=np.float)
  elif type == 'ER':
    #net = nx.erdos_renyi_graph(n, .006)
    targetDegree = 4.
    nEdgesPossible = ((n*n)-n)/2.
    pEdge = (n * targetDegree) / (2. * nEdgesPossible)
    assert(pEdge <= 1)

    # spare networks will likely be disconnected, so try a bunch
    tries = 100
    while tries > 0:
      net = nx.erdos_renyi_graph(n, pEdge)
      if nx.number_connected_components(net) > 1:
        tries = tries - 1
      else:
        break
    return net, nx.to_numpy_matrix(net, dtype=np.float)



In [ ]:
def ltProp(agents, adjMatrix, avgDegree=0, haltMin=.49, haltMax=.51, rs = None):
    # init some stuff for numba
    thresholds = np.zeros_like(nAgents)
    inp = np.zeros_like(nAgents)
    step = 0
    numNeighbors = np.zeros_like(nAgents)
    prevMean = 0.
    liveEdges = np.zeros_like(adjMatrix)
    pInfect = np.zeros_like(adjMatrix)
    flips = np.zeros_like(adjMatrix)
    
    globalThreshold = .5
    if rs is None:
        thresholds = globalThreshold * np.random.random((1, nAgents))
    else:
        thresholds = globalThreshold * rs.random((1, nAgents))
    
    numNeighbors = np.sum(adjMatrix, axis=0)
    prevMean = -1
    step = 1
    
    while not testThresh(agents, haltMin, haltMax) and (np.mean(agents) > prevMean):
        #while np.mean(agents) > prevMean:
        prevMean = np.mean(agents)
        inp = np.true_divide(np.dot(agents, adjMatrix), numNeighbors)
        agents = np.logical_or(agents, (inp >= thresholds)).astype(int)

        step = step + 1
        
    #print('LT-proportional stopped at step ' + str(step) + ' '+ str(np.mean(agents)))
    return agents

In [ ]:
def ltAbs(agents, adjMatrix, avgDegree=0, haltMin=.49, haltMax=.51, rs = None):
    # init some stuff for numba
    thresholds = np.zeros_like(nAgents)
    inp = np.zeros_like(nAgents)
    step = 0
    numNeighbors = np.zeros_like(nAgents)
    prevMean = 0.
    liveEdges = np.zeros_like(adjMatrix)
    pInfect = np.zeros_like(adjMatrix)
    flips = np.zeros_like(adjMatrix)

    # this controls the thresholds/pInfects for all contagion types
    globalThreshold = .5
    
    if rs is None:
        thresholds = np.random.randint(low=1, high=round(avgDegree*globalThreshold), size=(1, nAgents))
    else:
        thresholds = rs.randint(low=1, high=round(avgDegree*globalThreshold), size=(1, nAgents))
        
    numNeighbors = np.sum(adjMatrix, axis=0)
    #prevMean = -1
    step = 1
    
    while not testThresh(agents, haltMin, haltMax) and (np.mean(agents) > prevMean):
        prevMean = np.mean(agents)
        inp = np.dot(agents, adjMatrix)
        agents = np.logical_or(agents, (inp >= thresholds)).astype(int)

        step = step + 1
        
    print('LT-absolute stopped at step ' + str(step) + ' '+ str(np.mean(agents)))

    return agents

In [ ]:
def generateGraph(pp, batch_size = 1, random_state=None):
    gg = []
    print(pp)
    while True:
        nAgents = sidelength**2
        agents = np.zeros((1, nAgents))
        net, adjMatrix = genNet(nAgents, type='pref')

        for i in range(1): #make seeding more random, exp with neighbors
            #agents[0][1] = 1
            agents[0][random.randint(0, nAgents-1)] = 1.

        # seed neighbors of seeds
        for i in range(1):
            agents = np.logical_or(agents, np.dot(agents, adjMatrix)).astype(float) #all neighbors

        avgDegree = 2*net.number_of_edges() / float(net.number_of_nodes())

        if pp >= 0.5:
            agents_type = ltProp(agents, adjMatrix, avgDegree=avgDegree, haltMin=haltMin, haltMax=haltMax, rs = random_state)
        else:
            agents_type = ltAbs(agents, adjMatrix, avgDegree=avgDegree, haltMin=haltMin, haltMax=haltMax, rs = random_state)

        #if not testThresh(agents_type, haltMin, haltMax):
         #   print('bad data, LTabs1:\t'+str(np.mean(agents_type)))
       #     continue
        gparam = {}
        gparam['init'] = net
        gparam['graph'] = agents_type
        
        gg.append(gparam['graph'])
        if len(gg) >= batch_size:
            break

    return gg



In [ ]:
def compileG(ll, batch_size = 1, random_state=None):
    gall = []
    for g in ll:
        gall.append(generateGraph(g, batch_size, random_state))
    return gall

In [ ]:
ga = compileG([0.5])
gr_obs = ga[0] 

In [ ]:
print(gr_obs)

In [ ]:
prop_prob = elfi.Prior(scipy.stats.uniform, 0, 1)
print(type(prop_prob))

In [ ]:
# Y = elfi.Simulator(generateGraph, prop_prob, observed = gr_obs)
Y = elfi.Simulator(compileG, prop_prob, observed = gr_obs)

In [ ]:
def yMean(x):
    return np.mean(x)
s = elfi.Summary(yMean, Y)

In [ ]:
d = elfi.Distance('euclidean', s)

In [ ]:
elfi.draw(d)

In [ ]:
rej = elfi.Rejection(d, batch_size=1, seed=seed)

In [ ]:
N = 10
# You can give the sample method a `vis` keyword to see an animation how the prior transforms towards the
# posterior with a decreasing threshold.
%time result = rej.sample(N, quantile=0.1)

In [ ]:
result.samples['prop_prob'].mean()
